In [19]:
import numpy as np
import pandas as pd
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model
from keras.layers.merge import concatenate

In [3]:
def load_doc(filename):
    file = open(filename,'r')
    data = file.read()
    file.close()
    return data

In [4]:
def clean_doc(data):
    tokens = data.split()
    
    re_punct = re.compile('[%s]'%re.escape(string.punctuation))
    
    cleaned_text = [re_punct.sub('',w) for w in tokens]
    
    cleaned_text = [word for word in cleaned_text if word.isalpha()]
    
    stop_words =  stopwords.words('english')
    cleaned_text = [word for word in cleaned_text if word not in stop_words]
    
    cleaned_text = [word for word in cleaned_text if len(word) > 1]
    
    return cleaned_text
    

In [5]:
def add_doc_to_vocab(filepath, vocabulary):
    data = load_doc(filepath)
    clean_text = clean_doc(data)
    vocabulary.update(clean_text)

In [6]:
def process_docs(directory,vocabulary):
   
    for file in listdir(directory):
        if file.startswith('cv9'):
            continue     
      
        filepath = directory+'/'+file
      
        add_doc_to_vocab(filepath, vocabulary)

In [7]:
def process_vocabulary(min_occurences,vocabulary):
    tokens = [word for word,count in vocabulary.items() if count >= min_occurences]
    return tokens

In [8]:
def save_list(word_list, file_name):
    string = '\n'.join(word_list)
    file = open(file_name,'w')
    file.write(string)
    file.close()


In [9]:
vocabulary = Counter()
negative_reviews = 'txt_sentoken/neg'
positive_reviews = 'txt_sentoken/pos'
min_occurences = 2
reviews = [negative_reviews, positive_reviews]
for directory in reviews:
    process_docs(directory,vocabulary)

In [10]:
min_occurences = 2
tokens = process_vocabulary(min_occurences,vocabulary)
save_list(tokens, 'vocabulary_new_3.txt')

In [11]:
def clean_doc_vocab(data, vocabulary):
    tokens = data.split()
    
    re_punct = re.compile('[%s]'%re.escape(string.punctuation))
    
    cleaned_text = [re_punct.sub('',w) for w in tokens]
    
    cleaned_text = [word for word in cleaned_text if word in vocabulary]
    
    tokens = ' '.join(cleaned_text)
    
    return tokens 

In [12]:
def process_docs_2(directory, vocabulary, is_train):
    documents = []
    for file in listdir(directory):
        if is_train and file.startswith('cv9'):
            continue
        if not is_train and not file.startswith('cv9'):
            continue
        
        file_path = directory+'/'+file
        
        data = load_doc(file_path)
        
        cleaned_data = clean_doc_vocab(data,vocabulary)
        
        documents.append(cleaned_data)
        
    return documents

In [13]:
def load_clean_docs(vocabulary, is_train):
    negative_reviews = 'txt_sentoken/neg'
    positive_reviews = 'txt_sentoken/pos'
    
    neg_docs = process_docs_2(negative_reviews, vocabulary, is_train)
    pos_docs = process_docs_2(positive_reviews, vocabulary, is_train)
    
    docs = neg_docs + pos_docs
    
    labels = np.array([0 for _ in range(len(neg_docs))] +  [1 for _ in range(len(pos_docs))])
    
    return docs, labels

In [14]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [15]:
def encode_pad(tokenizer, docs, max_len):
   
    encoded = tokenizer.texts_to_sequences(docs)
    padded_sequence = pad_sequences(encoded, maxlen = max_len, padding = 'post')
    return padded_sequence

In [16]:
is_train = True
vocabulary = load_doc('vocabulary_new_3.txt')
voacbulary = set(vocabulary.split())
train_docs,train_labels = load_clean_docs(vocabulary, is_train)
review_len = [len(item.split()) for item in train_docs]
max_len = max(review_len)
tokenizer = create_tokenizer(train_docs)
X_train = encode_pad(tokenizer, train_docs,max_len)

In [17]:
is_train =  False
test_docs,test_labels = load_clean_docs(vocabulary, is_train)
X_test= encode_pad(tokenizer, test_docs, max_len)

In [18]:
vocabulary_size = len(tokenizer.word_index)+1
output_dimension_size = 100
input_sequence_length = max_len

In [71]:
def make_model(input_sequnce, voabulary_length):
    input1 = Input(shape = (input_sequnce,))
    embedding = Embedding(voabulary_length, 100)(input1)
    cnn1 = Conv1D(32, 4, activation = 'relu')(embedding)
    dropout = Dropout(0.2)(cnn1)
    max_pool1 = MaxPooling1D()(dropout)
    flatten1 = Flatten()(max_pool1)
    
    input2 = Input(shape = (input_sequnce,))
    embedding = Embedding(voabulary_length, 100)(input2)
    cnn2 = Conv1D(32, 6, activation = 'relu')(embedding)
    dropout = Dropout(0.2)(cnn2)
    max_pool2 = MaxPooling1D()(dropout)
    flatten2 = Flatten()(max_pool2)
   
    input3 = Input(shape = (input_sequnce,))
    embedding = Embedding(voabulary_length, 100)(input3)
    cnn3 = Conv1D(32, 8, activation = 'relu')(embedding)
    dropout = Dropout(0.2)(cnn3)
    max_pool3 = MaxPooling1D()(dropout)
    flatten3 = Flatten()(max_pool3)

    
    merged = concatenate([flatten1, flatten2, flatten3])
    
    dense1 = Dense(20, activation = 'relu')(merged)
    dropout = Dropout(0.2)(dense1)
    dense2 = Dense(10, activation = 'relu')(dropout)
    
    outputs = Dense(1, activation = 'sigmoid')(dense2)
    model = Model(inputs = [input1,input2,input3], outputs = outputs)
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [72]:
model = make_model(input_sequence_length, vocabulary_size)

In [73]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 2282)         0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, 2282)         0                                            
__________________________________________________________________________________________________
input_35 (InputLayer)           (None, 2282)         0                                            
__________________________________________________________________________________________________
embedding_33 (Embedding)        (None, 2282, 100)    2767500     input_33[0][0]                   
____________________________________________________________________________________________

In [74]:
results = pd.DataFrame()
accuracy = []
batches = []
epochs = []
for b_size in [16,32,64]:
    for epoch in [10,20,30]:
        model.fit([X_train, X_train, X_train], train_labels,batch_size = b_size, epochs = epoch)
        _,acc = model.evaluate([X_test, X_test, X_test], test_labels)
        accuracy.append(acc)
        batches.append(b_size)
        epochs.append(epoch)

results['Batch'] = batches
results['Epochs'] = epochs
results['Accuracy'] = accuracy

Epoch 1/10
1800/1800 [==============================] - 116s 65ms/step - loss: 0.6957 - accuracy: 0.5033
Epoch 2/10
1800/1800 [==============================] - 100s 56ms/step - loss: 0.6508 - accuracy: 0.6156
Epoch 3/10
1800/1800 [==============================] - 85s 47ms/step - loss: 0.2885 - accuracy: 0.8783
Epoch 4/10
1800/1800 [==============================] - 83s 46ms/step - loss: 0.0553 - accuracy: 0.9750
Epoch 5/10
1800/1800 [==============================] - 83s 46ms/step - loss: 0.0274 - accuracy: 0.9833
Epoch 6/10
1800/1800 [==============================] - 83s 46ms/step - loss: 0.0237 - accuracy: 0.9906
Epoch 7/10
1800/1800 [==============================] - 84s 46ms/step - loss: 0.0177 - accuracy: 0.9933
Epoch 8/10
1800/1800 [==============================] - 84s 47ms/step - loss: 0.0163 - accuracy: 0.9956
Epoch 9/10
1800/1800 [==============================] - 90s 50ms/step - loss: 0.0350 - accuracy: 0.9906
Epoch 10/10
200/200 [==============================] - 2s 9ms/

1800/1800 [==============================] - 377s 210ms/step - loss: 0.0031 - accuracy: 0.9972
Epoch 7/20
1800/1800 [==============================] - 528s 293ms/step - loss: 0.0066 - accuracy: 0.9961
Epoch 8/20
1800/1800 [==============================] - 246s 137ms/step - loss: 0.0065 - accuracy: 0.9961
Epoch 9/20
1800/1800 [==============================] - 954s 530ms/step - loss: 0.0063 - accuracy: 0.9933
Epoch 10/20
1800/1800 [==============================] - 229s 127ms/step - loss: 0.0049 - accuracy: 0.9983
Epoch 11/20
1800/1800 [==============================] - 752s 418ms/step - loss: 0.0046 - accuracy: 0.9972
Epoch 12/20
1800/1800 [==============================] - 603s 335ms/step - loss: 0.0043 - accuracy: 0.9961
Epoch 13/20
1800/1800 [==============================] - 4704s 3s/step - loss: 0.0082 - accuracy: 0.9917
Epoch 14/20
1800/1800 [==============================] - 83s 46ms/step - loss: 0.0054 - accuracy: 0.9961
Epoch 15/20
1800/1800 [==============================] -

1800/1800 [==============================] - 65s 36ms/step - loss: 0.0050 - accuracy: 0.9978
Epoch 3/30
1800/1800 [==============================] - 72s 40ms/step - loss: 0.0034 - accuracy: 0.9983
Epoch 4/30
1800/1800 [==============================] - 68s 38ms/step - loss: 0.0052 - accuracy: 0.9983
Epoch 5/30
1800/1800 [==============================] - 61s 34ms/step - loss: 0.0065 - accuracy: 0.9961
Epoch 6/30
1800/1800 [==============================] - 62s 34ms/step - loss: 0.0050 - accuracy: 0.9967
Epoch 7/30
1800/1800 [==============================] - 64s 36ms/step - loss: 0.0077 - accuracy: 0.9944
Epoch 8/30
1800/1800 [==============================] - 69s 38ms/step - loss: 0.0071 - accuracy: 0.9939
Epoch 9/30
1800/1800 [==============================] - 67s 37ms/step - loss: 0.0057 - accuracy: 0.9972
Epoch 10/30
1800/1800 [==============================] - 63s 35ms/step - loss: 0.0070 - accuracy: 0.9950
Epoch 11/30
1800/1800 [==============================] - 61s 34ms/step - l

In [76]:
results.sort_values(by = 'Accuracy', ascending = False)

,Batch,Epochs,Accuracy
0,16,10,0.885
1,16,20,0.860
3,32,10,0.845
2,16,30,0.840
4,32,20,0.840
5,32,30,0.830
7,64,20,0.820
8,64,30,0.820
6,64,10,0.815


In [34]:
model = load_model('model_cnn.h5')

In [35]:
model.evaluate(X_test, test_labels)[1]

200/200 [==============================] - 1s 5ms/step


0.8899999856948853